In [1]:
from openai import AzureOpenAI, OpenAI
import os
import sys
import fitz 
import yaml 
from datetime import datetime
from utils import *

N = 2
config_wm_file = "configs/wm_outside.yaml"
config_test_file = "configs/reviewer.yaml"
tmp_dir = "./tmp/"
input_dir = "./ICLR2025_accepted/pdf/"
results_dir = "./results/"

model_name = "gemini-2.5-flash"
client = OpenAI(api_key=os.environ["GCP_API_KEY"],
                base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

with open(config_wm_file, 'r') as file:
    config_wm_list = yaml.safe_load(file)
    print(config_wm_list)
with open(config_test_file, 'r') as file:
    config_test_list = yaml.safe_load(file)
    print(config_test_list)
    

for i in range(N):
    print(f"i={i}")
    input_path = input_dir + str(i) + '.pdf'

    for config_wm in config_wm_list:
        if config_wm['type'] == 'intext':
            c = add_abstract_blend_wm(input_path, client, model_name, config_wm)
            for config_test in config_test_list:
                res = test_content(c, client, model_name, config_test)
                save_results(results_dir + str(i) + f"_{timestamp}_expt.yaml", model_name, res, config_wm, config_test)         
        elif config_wm['type'] == 'outside':
            out_fname = add_wm(input_path, tmp_dir, config_wm)
            print('Output filename:', out_fname)
            for config_test in config_test_list:
                res = test(out_fname, client, model_name, extract_text, config_test)
                save_results(results_dir + str(i) + f"_{timestamp}_expt.yaml", model_name, res, config_wm, config_test)         
        else:
            print('****** Error: Unknown WM type ********')

            

ModuleNotFoundError: No module named 'openai'